In [1]:
from unityagents import UnityEnvironment
import numpy as np

In [2]:
env = UnityEnvironment(file_name="/home/sumuk/rl/deep-reinforcement-learning/p1_navigation/Banana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print(brain)

Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


In [5]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


In [6]:
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import torch 
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.distributions import Categorical
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)
class Actor(nn.Module):
    def __init__(self,state_size,action_size):
        super(Actor,self).__init__()
        self.state_size=state_size
        self.action_size=action_size
        self.fc1 = nn.Linear(self.state_size,128)
        self.fc3 = nn.Linear(128,64)
        self.fc2 = nn.Linear(64,self.action_size)
        self.entropy=[]
#         self.fc3.weight.data.uniform_(-1e-2,1e-2)
#         self.fc2.weight.data.uniform_(-1e-2,1e-2)
#         self.fc1.weight.data.uniform_(-1e-2,1e-2)
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc3.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc2.weight.data.uniform_(-1e-3,1e-3)
    def forward(self,state):
        y = f.relu(self.fc1(state))
        y = f.relu(self.fc3(y))
        y = f.softmax(self.fc2(y),dim=-1)
        return y
    def act(self,state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        self.entropy.append(m.entropy().detach().numpy())
        return action.item(), m.log_prob(action)

class Critic(nn.Module):
    def __init__(self,state_size):
        super(Critic,self).__init__()
        self.state_size = state_size
        self.fc1 = nn.Linear(self.state_size,128)
        self.fc3 = nn.Linear(128,64)
        self.fc2 = nn.Linear(64,1)
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc3.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc2.weight.data.uniform_(-3e-3,3e-3)
    def forward(self,state):
        state = torch.from_numpy(state).float().to(device)
        y = f.relu(self.fc1(state))
        y = f.relu(self.fc3(y))
        y = self.fc2(y)
        return y.cpu()

In [7]:
import torch.optim as optim
def exp_lr_scheduler(optimizer,decay):
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
        
    lr = lr * decay
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer,lr
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
actor = Actor(37,4).to(device)
critic = Critic(37).to(device)
optim_actor = optim.Adam(actor.parameters(), lr=1e-3)
optim_critic = optim.Adam(critic.parameters(),lr=1e-3)

def reinforce(n_episodes=1000, max_t=1000, gamma=0.95, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        states=[]
        next_states=[]
        score_list=[]
        while(True):
#         for _ in range(5):
            state_list=[]
            log_prob_list=[]
            reward_list=[]
            env_info = env.reset(train_mode=True)[brain_name]
            state =  env_info.vector_observations[0] 
            state_list.append(state)
            for t in range(max_t):
                action, log_prob = actor.act(state)
                log_prob_list.append(log_prob)
                env_info = env.step(action)[brain_name] 
                state, reward, done = env_info.vector_observations[0],env_info.rewards[0],env_info.local_done[0]
                state_list.append(state)
                reward_list.append(reward)
                if done:
                    break
            saved_log_probs.extend(log_prob_list)
            states.extend(state_list[:-1])
            next_states.extend(state_list[1:])
            r=[0.0]
            for i in reversed(reward_list):
                r.append(i+gamma*r[-1])
            r=r[:0:-1]
            #r = (r - np.mean(r))/(1e-5+np.std(r))
            rewards.extend(r)
            score_list.append(np.sum(reward_list))
            if(len(states)>1024):
                break
        scores_deque.append(np.mean(score_list))
        scores.append(np.mean(score_list))
       
        if(i_episode%1==0):
            for _ in range(5): 
                vstate = critic(np.array(states)).squeeze(-1)
                vstatep = critic(np.array(next_states)).squeeze(-1)
                vlabel= torch.tensor(rewards) + vstatep
                critic_loss = f.mse_loss(vstate,vlabel)
                optim_critic.zero_grad()
                critic_loss.backward()
                torch.nn.utils.clip_grad_norm_(critic.parameters(), 1)
                optim_critic.step()
        baseline = critic(np.array(states)).detach().squeeze(-1).numpy()
        baseline = (baseline - np.mean(baseline))/(np.std(baseline)+1e-5)*np.std(rewards)+np.mean(rewards)
        adv = rewards - baseline
        adv = (adv - np.mean(adv))/(1e-5+np.std(adv)) 
        policy_loss = []
        for log_prob,A in zip(saved_log_probs,adv):
            policy_loss.append(-log_prob * (A))
        policy_loss = torch.cat(policy_loss).sum()/len(score_list)
        optim_actor.zero_grad()
        policy_loss.backward()
        torch.nn.utils.clip_grad_norm_(actor.parameters(), 5)
        
        optim_actor.step()

           
            
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=13.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores
        
scores=reinforce()

Episode 100	Average Score: 0.64
Episode 200	Average Score: 9.64
Episode 300	Average Score: 12.19
Episode 400	Average Score: 11.78
Episode 500	Average Score: 11.89
Episode 600	Average Score: 12.63
Episode 700	Average Score: 12.62
Episode 800	Average Score: 12.78
Environment solved in 712 episodes!	Average Score: 13.01


In [ ]:
import time
for k in range(1):
    scores = []
    for i in range(10):
        env_info = env.reset(train_mode=False)[brain_name] 
        state = env_info.vector_observations[0] 
        score = 0
        for j in range(1000):
            action, log_prob = actor.act(state)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]
            state = next_state
            score += reward
            #print("epoch {0} score {1}".format(j,score))
            time.sleep(.1)
            if done:
                break
        scores.append(score)
    print(np.mean(scores))            
